In [1]:
#!/usr/bin/env python

# This example shows how to construct a surface from a point cloud.
# First we generate a volume using the
# vtkSurfaceReconstructionFilter. The volume values are a distance
# field. Once this is generated, the volume is countoured at a
# distance value of 0.0.

import os
import string
import numpy as np
import vtk
from vtk.util.misc import vtkGetDataRoot
VTK_DATA_ROOT = vtkGetDataRoot()

In [2]:
ordering = [0, 1, 2, 13, 24, 35, 46, 57, 68, 79, 90, 101, 112, 113, 124, 135, 146, 157, 223, 334, 394, 405, 416, 427, 438, 3, 4, 5, 6, 7, 8, 9, 250, 251, 252, 253, 254, 255, 256, 258, 259, 260, 261, 262, 263, 264, 410, 411, 412, 413, 414, 415, 417, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 226, 227, 228, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 247, 248, 249, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 204, 205, 206, 207, 208, 209, 210, 397, 398, 399, 400, 401, 402, 403, 404, 406, 407, 408, 409, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 103, 104, 105, 106, 107, 108, 109, 110, 111, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 125, 126, 345, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 369, 370, 371, 89, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 127, 128, 129, 130, 131, 132, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 161, 162, 163, 282, 283, 284, 285, 286, 287, 288, 289, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 324, 325, 372, 373, 374, 375, 376, 377, 378, 380, 381, 382, 383, 384, 385, 386, 387, 388, 418, 419, 420, 421, 422, 423, 424, 425, 426, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 326, 327, 328, 329, 330, 331, 332, 333, 336, 337, 338, 339, 340, 341, 342, 343, 344, 168, 179, 190, 201, 212, 224, 235, 246, 257, 268, 279, 290, 301, 312, 323, 335, 346, 357, 368, 379, 389, 390, 391, 392, 393, 395, 396, 211, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 225, 265, 266, 267, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 280, 281]
print(len(ordering))

449


In [3]:
def rearrangeWithOrder(order, X):
    
    Z = list()
    for e in order:
        Z.append(X[e])
    return np.array(Z)

In [4]:
# X = np.load('MasksForPlot.npy')
X = np.load('MasksNPYFiles/npyFiles005/masksFinalContoured.npy')
# Y = X[:,:,:,0]
# Y = X[0:5,:,:,0]
# X = rearrangeWithOrder(ordering, X)
Y = X[:,:,:]          #Set frames here
# Y = X
pts = np.argwhere(Y*1)
print(X.shape, X.size, Y.shape, Y.size)
print(len(pts.tolist()))
print(pts.tolist()[0:5])

(449, 384, 384) 66207744 (449, 384, 384) 66207744
20738
[[0, 114, 179], [0, 114, 180], [0, 114, 181], [0, 114, 187], [0, 114, 188]]


In [5]:
# Read some points. Use a programmable filter to read them.
pointSource = vtk.vtkProgrammableSource()

def readPoints():
    output = pointSource.GetPolyDataOutput()
    points = vtk.vtkPoints()
    output.SetPoints(points)

    file = open(os.path.normpath(os.path.join(VTK_DATA_ROOT, "Data/cactus.3337.pts")))

    line = file.readline()
    while line:
        data = line.split()
        if data and data[0] == 'p':
            x, y, z = float(data[1]), float(data[2]), float(data[3])
            points.InsertNextPoint(x, y, z)
        line = file.readline()
        
def gen_points():
    output = pointSource.GetPolyDataOutput()
    points = vtk.vtkPoints()
    output.SetPoints(points)

#     file = open(os.path.normpath(os.path.join(VTK_DATA_ROOT, "Data/cactus.3337.pts")))

#     line = file.readline()
    pt = pts.tolist()

    for point in pt:
#         data = line.split()
#         if data and data[0] == 'p':
#             x, y, z = float(data[1]), float(data[2]), float(data[3])
        points.InsertNextPoint(point)
#         line = file.readline()

pointSource.SetExecuteMethod(gen_points)


# Construct the surface and create isosurface.
surf = vtk.vtkSurfaceReconstructionFilter()
surf.SetInputConnection(pointSource.GetOutputPort())

# print(type(pointSource.GetOutputPort()))

# cf = vtk.vtkContourFilter()
# cf.SetInputConnection(surf.GetOutputPort())
# cf.SetValue(0, 0.0)

# print(type(surf.GetOutputPort()))
# Sometimes the contouring algorithm can create a volume whose gradient
# vector and ordering of polygon (using the right hand rule) are
# inconsistent. vtkReverseSense cures this problem.
# reverse = vtk.vtkReverseSense()
# reverse.SetInputConnection(cf.GetOutputPort())
# reverse.ReverseCellsOn()
# reverse.ReverseNormalsOn()

map = vtk.vtkPolyDataMapper()
# map.SetInputConnection(reverse.GetOutputPort())
map.SetInputConnection(surf.GetOutputPort())
map.ScalarVisibilityOff()

surfaceActor = vtk.vtkActor()
surfaceActor.SetMapper(map)
surfaceActor.GetProperty().SetDiffuseColor(1.0000, 0.3882, 0.2784)
surfaceActor.GetProperty().SetSpecularColor(1, 1, 1)
surfaceActor.GetProperty().SetSpecular(.4)
surfaceActor.GetProperty().SetSpecularPower(5)

# Create the RenderWindow, Renderer and both Actors
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# Add the actors to the renderer, set the background and size
ren.AddActor(surfaceActor)
ren.SetBackground(1, 1, 1)
renWin.SetSize(400, 400)
ren.GetActiveCamera().SetFocalPoint(0, 0, 0)
ren.GetActiveCamera().SetPosition(1, 0, 0)
ren.GetActiveCamera().SetViewUp(0, 0, 1)
ren.ResetCamera()
ren.GetActiveCamera().Azimuth(20)
ren.GetActiveCamera().Elevation(30)
ren.GetActiveCamera().Dolly(1.2)
ren.ResetCameraClippingRange()

iren.Initialize()
renWin.Render()
iren.Start()
